## Harmonic Oscillator

$$m\frac{d^2u}{dt^2} + \mu\frac{du}{dt} + k u =0,\quad u(0)=1,\quad \frac{du}{dt}(0) = 0,\quad t\in[0, 1]$$

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        
def to_tensor(x):
    return torch.tensor(x).float().view(-1, 1).to(device)

def to_array(x):
    return x.view(-1).cpu().detach().numpy()

def gradient(y, x):
    return torch.autograd.grad(y, x, grad_outputs=torch.ones_like(y),
                               create_graph=True, retain_graph=True)[0]

In [15]:
## Equation parameters:
d, w0 = 2, 20       ## reference value w0 = 20
mu, k = 2*d, w0**2

t_min, t_max, t_size = 0, 1, 101
t_test = np.linspace(t_min, t_max, t_size)       # (t_size,)

n_pde = 101
## Domain: u_tt + mu * u_t + k * u
t_pde = np.random.rand(n_pde)*(t_max - t_min) + t_min
t_pde = to_tensor(t_pde)

t_ic = to_tensor(0)
u_ic = to_tensor(0)
du_ic = to_tensor(1)

In [12]:
class PINN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(1, 50),  nn.Tanh(),
            nn.Linear(50, 50), nn.Tanh(),
            nn.Linear(50, 50), nn.Tanh(),
            nn.Linear(50, 1),
        )

    def forward(self, t):
        return self.net(t)  # (N, 1)
    
    def residual_loss(self, t, mu=mu, k=k):
        t.requires_grad = True
        u = self.forward(t)
        u_t = gradient(u, t)
        u_tt = gradient(u_t, t)
        residual = u_tt + mu * u_t + k * u
        return torch.mean(residual**2)
    
    def ic_du_loss(self, t):
        t.requires_grad = True
        u = self.forward(t)
        u_t = gradient(u, t)
        return torch.mean((u_t - torch.full_like(t, 0))**2)
    
    def ic_u_loss(self, t):
        u = self.forward(t)
        return torch.mean((u - torch.full_like(t, 1))**2)

In [14]:
## Training
n_epochs = 1000
learning_rate = 1e-3

model = PINN().to(device)
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.985)

for epoch in range(1, n_epochs + 1):
    model.train()
    loss_pde = model.residual_loss(t_pde)
    loss_ic_du = model.ic_du_loss(t_ic)
    loss_ic_u = model.ic_u_loss(t_ic)

    loss = loss_pde * 1e-4 + loss_ic_du * 1e-2 + loss_ic_u
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    scheduler.step()

    if epoch % (n_epochs // 10) == 0:
        print(f"[{epoch:5d}/{n_epochs}] (lr: {scheduler.get_last_lr()[0]:.2e}) "
              f"loss: {loss.item():.2e} "
              f"(pde: {loss_pde.item():.2e} ic_du: {loss_ic_du.item():.2e} "
              f"ic_u: {loss_ic_u.item():.2e})")

[  100/1000] (lr: 1.00e-03) loss: 6.75e-01 (pde: 1.46e+03 ic_du: 8.78e-01 ic_u: 5.21e-01)
[  200/1000] (lr: 1.00e-03) loss: 6.22e-01 (pde: 2.05e+03 ic_du: 3.25e+00 ic_u: 3.85e-01)
[  300/1000] (lr: 1.00e-03) loss: 5.45e-01 (pde: 1.83e+03 ic_du: 5.09e+00 ic_u: 3.10e-01)
[  400/1000] (lr: 1.00e-03) loss: 3.41e-01 (pde: 1.57e+03 ic_du: 2.25e+00 ic_u: 1.62e-01)
[  500/1000] (lr: 1.00e-03) loss: 1.81e-01 (pde: 1.38e+03 ic_du: 9.60e-02 ic_u: 4.27e-02)
[  600/1000] (lr: 1.00e-03) loss: 1.68e-01 (pde: 1.32e+03 ic_du: 6.13e-02 ic_u: 3.58e-02)
[  700/1000] (lr: 1.00e-03) loss: 1.49e-01 (pde: 1.19e+03 ic_du: 5.31e-02 ic_u: 2.90e-02)
[  800/1000] (lr: 1.00e-03) loss: 1.16e-01 (pde: 9.65e+02 ic_du: 3.52e-02 ic_u: 1.92e-02)
[  900/1000] (lr: 1.00e-03) loss: 7.88e-02 (pde: 6.94e+02 ic_du: 1.01e-02 ic_u: 9.35e-03)
[ 1000/1000] (lr: 9.85e-04) loss: 5.16e-02 (pde: 4.73e+02 ic_du: 6.86e-04 ic_u: 4.25e-03)


In [ ]:
## Evaluation
with torch.no_grad():
    t_test = to_tensor(t_test)
    u_pred = model(t_test)

U_pred = to_array(u_pred).flatten()

